In [238]:
import os
import pandas as pd

# Data cleaning

Before using this notebook, you will need to have used the `nem-data` library to download data - see the instructions for this here

Cleaning data requires two tasks
- identification
- cleaning

See what files you have downloaded:

In [239]:
home = os.path.expanduser('~')
nem_files = os.listdir(os.path.join(home, 'nem-data', 'demand'))

nem_files

['2018-01', '2018-02']

If we look at a specific month, we see that `nem-data` has downloaded a few files for each month:

In [240]:
nem_files = os.listdir(os.path.join(home, 'nem-data', 'demand', '2018-01'))

nem_files

['clean.csv',
 'DISPATCHREGIONSUM.zip',
 'PUBLIC_DVD_DISPATCHREGIONSUM_201801010000.CSV']

Let's grab `clean.csv` and extract a potential target.

The data provided for the NEM is large - making working on problems in the NEM great for data scientists:

In [241]:
raw = pd.read_csv(os.path.join(home, 'nem-data', 'demand', '2018-01', 'clean.csv'), index_col=4, parse_dates=True)
raw.head()

,I,DISPATCH,REGIONSUM,4,RUNNO,REGIONID,DISPATCHINTERVAL,INTERVENTION,TOTALDEMAND,AVAILABLEGENERATION,...,LOWER60SECACTUALAVAILABILITY,LOWER5MINACTUALAVAILABILITY,LOWERREGACTUALAVAILABILITY,LORSURPLUS,LRCSURPLUS,TOTALINTERMITTENTGENERATION,DEMAND_AND_NONSCHEDGEN,UIGF,SEMISCHEDULE_CLEAREDMW,SEMISCHEDULE_COMPLIANCEMW
SETTLEMENTDATE,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:05:00,D,DISPATCH,REGIONSUM,4.0,1.0,NSW1,2.017123e+10,0.0,7021.71,11526.264,...,824.000000,687.64070,227.665483,NaN,NaN,0.17736,7042.60736,89.264,89.264,0.0
2018-01-01 00:05:00,D,DISPATCH,REGIONSUM,4.0,1.0,QLD1,2.017123e+10,0.0,6057.98,10651.650,...,200.000000,35.00000,189.300009,NaN,NaN,96.21874,6167.23874,0.650,0.650,0.0
2018-01-01 00:05:00,D,DISPATCH,REGIONSUM,4.0,1.0,SA1,2.017123e+10,0.0,1311.69,2097.706,...,94.000000,92.83001,65.000000,NaN,NaN,56.26510,1382.86510,232.706,232.706,0.0
2018-01-01 00:05:00,D,DISPATCH,REGIONSUM,4.0,1.0,TAS1,2.017123e+10,0.0,981.16,2521.608,...,257.112576,151.07581,18.000000,NaN,NaN,116.82656,1097.98656,153.608,153.608,0.0
2018-01-01 00:05:00,D,DISPATCH,REGIONSUM,4.0,1.0,VIC1,2.017123e+10,0.0,4306.37,8726.033,...,330.000000,250.00000,162.662354,NaN,NaN,148.25500,4468.83500,202.033,202.033,0.0


In [242]:
region = raw['REGIONID'] == 'SA1'
cols = ['REGIONID', 'TOTALDEMAND']

target = raw[region][cols]
target.head()

,REGIONID,TOTALDEMAND
SETTLEMENTDATE,,
2018-01-01 00:05:00,SA1,1311.69
2018-01-01 00:10:00,SA1,1300.79
2018-01-01 00:15:00,SA1,1288.66
2018-01-01 00:20:00,SA1,1269.00
2018-01-01 00:25:00,SA1,1289.67


## Exercise - raw dataset health check

On the `raw` dataframe, check
- how many missing values we have in each column
- check for duplicates

On the `target` dataframe
- check the integrity of the time stamps (do we have any gaps?)

In [243]:
raw.isnull().sum()

I                                   0
DISPATCH                            0
REGIONSUM                           0
4                                   0
RUNNO                               0
REGIONID                            0
DISPATCHINTERVAL                    0
INTERVENTION                        0
TOTALDEMAND                         0
AVAILABLEGENERATION                 0
AVAILABLELOAD                       0
DEMANDFORECAST                      0
DISPATCHABLEGENERATION              0
DISPATCHABLELOAD                    0
NETINTERCHANGE                      0
EXCESSGENERATION                    0
LOWER5MINDISPATCH               51480
LOWER5MINIMPORT                 51480
LOWER5MINLOCALDISPATCH              0
LOWER5MINLOCALPRICE             51480
LOWER5MINLOCALREQ               51480
LOWER5MINPRICE                  51480
LOWER5MINREQ                    51480
LOWER5MINSUPPLYPRICE            51480
LOWER60SECDISPATCH              51480
LOWER60SECIMPORT                51480
LOWER60SECLO

Some of the columns on the raw dataset contain no information at all (100% null values), but we are going to focus our analysis on `TOTALDEMAND`, which thankfully has no missing values at all.

In [244]:
raw.duplicated().sum()

0

The raw dataset has no duplicated rows.

In [245]:
td = pd.Timedelta(minutes = 5)

(target.index.to_series().diff() != td).sum()


1369

Some rows on the target dataset appear to have time deltas that are different from the typical 5 minutes. We are going to check if there are any duplicate entries.

In [246]:
target = target[target.index.duplicated(keep='first') == False]

(target.index.to_series().diff() != td).sum()

1

After dropping the duplicated rows, there's only a single row with a time delta that isn't 5 minutes. This is normal, because the first row can't be compared to the previous.